### Unsloth

In [ ]:
import os
import torch
import torchvision
import unsloth
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login
from peft import LoraConfig, get_peft_model
import torch
from unsloth import FastLanguageModel
import torch
from unsloth.chat_templates import get_chat_template
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only
import random
from typing import List, Tuple
from unsloth.chat_templates import get_chat_template

login(token="Removed")

# Prepend CUDA bin and lib64 directories to PATH and LD_LIBRARY_PATH
cuda_bin = "/nas/longleaf/rhel8/apps/cuda/11.8/bin"
cuda_lib = "/nas/longleaf/rhel8/apps/cuda/11.8/lib64"

# Update PATH
os.environ["PATH"] = cuda_bin + ":" + os.environ.get("PATH", "")

# Update LD_LIBRARY_PATH
os.environ["LD_LIBRARY_PATH"] = cuda_lib + ":" + os.environ.get("LD_LIBRARY_PATH", "")

### Model Setup

In [91]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-PCIE-40GB. Num GPUs = 1. Max memory: 39.495 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu118. CUDA: 8.0. CUDA Toolkit: 11.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.61it/s]


In [92]:

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

In [33]:
import random
from typing import List, Tuple
import json

def load_chat_dataset(input_path):
    """
    Load a chat-style message dataset from a JSON or JSONL file.
    
    Returns:
        data (list): A list of message dictionaries.
    """
    with open(input_path, "r", encoding="utf-8") as f:
        if input_path.endswith(".jsonl"):
            data = [json.loads(line) for line in f]
        else:  # Assume .json
            data = json.load(f)
    
    print(f"Dataset loaded from: {input_path}")
    print(f"Total examples: {len(data)}")
    return data


def train_test_split(member: str, test_size: float = 0.2, seed: int = 42, data_path: str = '/work/users/s/m/smerrill/Albemarle/dataset') -> Tuple[List[dict], List[dict]]:
    """
    Splits the dataset into training and test sets. Synthetic data is always added to the training set.

    Parameters:
    - member: The name identifier for the board member.
    - test_size: Proportion of the real (non-synthetic) data to include in the test split.
    - seed: Random seed for reproducibility.
    - data_path: Base directory for the dataset files.

    Returns:
    - A tuple (train_data, test_data)
    """
    real_data, synth_data = [], []

    if member == 'acuff':
        real_data = load_chat_dataset(os.path.join(data_path, 'kateacuff.txt'))
        synth_data = load_chat_dataset(os.path.join(data_path, 'synth_kateacuff.txt'))
    elif member == 'osborne':
        real_data = load_chat_dataset(os.path.join(data_path, 'ellenosborne.txt'))
        synth_data = load_chat_dataset(os.path.join(data_path, 'synth_ellenosborne.txt'))
    elif member == 'paige':
        real_data = load_chat_dataset(os.path.join(data_path, 'grahampaige.txt'))
        synth_data = load_chat_dataset(os.path.join(data_path, 'synth_grahampaige.txt'))
    elif member == 'le':
        real_data = load_chat_dataset(os.path.join(data_path, 'judyle.txt'))
        synth_data = load_chat_dataset(os.path.join(data_path, 'synth_judyle.txt'))
    elif member == 'callsen':
        real_data = load_chat_dataset(os.path.join(data_path, 'katrinacallsen.txt'))
    elif member == 'oberg':
        real_data = load_chat_dataset(os.path.join(data_path, 'davidoberg.txt'))
    elif member == 'alcaro':
        real_data = load_chat_dataset(os.path.join(data_path, 'jonnoalcaro.txt'))
    else:
        raise ValueError(f"Unknown member: {member}")

    if not 0 < test_size < 1:
        raise ValueError("test_size must be a float between 0 and 1.")

    # Shuffle and split only the real data
    random.seed(seed)
    shuffled_real = real_data.copy()
    random.shuffle(shuffled_real)

    split_index = int(len(shuffled_real) * (1 - test_size))
    train_data = shuffled_real[:split_index] + synth_data
    test_data = shuffled_real[split_index:]

    return train_data, test_data

train_data, test_data = train_test_split('acuff')

In [48]:
from datasets import Dataset
dataset2 = Dataset.from_list(train_data)
dataset2 = dataset2.map(formatting_prompts_func2, batched = True,)

Map: 100%|██████████| 296/296 [00:00<00:00, 16145.36 examples/s]


In [49]:
dataset2

Dataset({
    features: ['messages', 'text'],
    num_rows: 296
})

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [26]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [27]:
dataset['conversations'][0]

[{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.',
  'role': 'user'},
 {'content': 'Boolean operators a

In [28]:
dataset['text'][0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nExplain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code t

In [9]:
dataset

Dataset({
    features: ['conversations', 'source', 'score', 'text'],
    num_rows: 100000
})

### Dataset

In [104]:
def load_chat_dataset(input_path):
    """
    Load a chat-style message dataset from a JSON or JSONL file.
    
    Returns:
        data (list): A list of message dictionaries.
    """
    with open(input_path, "r", encoding="utf-8") as f:
        if input_path.endswith(".jsonl"):
            data = [json.loads(line) for line in f]
        else:  # Assume .json
            data = json.load(f)
    
    print(f"Dataset loaded from: {input_path}")
    print(f"Total examples: {len(data)}")
    return data


def train_test_split(member: str, test_size: float = 0.2, seed: int = 42, data_path: str = '/work/users/s/m/smerrill/Albemarle/dataset') -> Tuple[List[dict], List[dict]]:
    """
    Splits the dataset into training and test sets. Synthetic data is always added to the training set.

    Parameters:
    - member: The name identifier for the board member.
    - test_size: Proportion of the real (non-synthetic) data to include in the test split.
    - seed: Random seed for reproducibility.
    - data_path: Base directory for the dataset files.

    Returns:
    - A tuple (train_data, test_data)
    """
    real_data, synth_data = [], []

    if member == 'acuff':
        real_data = load_chat_dataset(os.path.join(data_path, 'kateacuff.txt'))
        synth_data = load_chat_dataset(os.path.join(data_path, 'synth_kateacuff.txt'))
    elif member == 'osborne':
        real_data = load_chat_dataset(os.path.join(data_path, 'ellenosborne.txt'))
        synth_data = load_chat_dataset(os.path.join(data_path, 'synth_ellenosborne.txt'))
    elif member == 'paige':
        real_data = load_chat_dataset(os.path.join(data_path, 'grahampaige.txt'))
        synth_data = load_chat_dataset(os.path.join(data_path, 'synth_grahampaige.txt'))
    elif member == 'le':
        real_data = load_chat_dataset(os.path.join(data_path, 'judyle.txt'))
        synth_data = load_chat_dataset(os.path.join(data_path, 'synth_judyle.txt'))
    elif member == 'callsen':
        real_data = load_chat_dataset(os.path.join(data_path, 'katrinacallsen.txt'))
    elif member == 'oberg':
        real_data = load_chat_dataset(os.path.join(data_path, 'davidoberg.txt'))
    elif member == 'alcaro':
        real_data = load_chat_dataset(os.path.join(data_path, 'jonnoalcaro.txt'))
    else:
        raise ValueError(f"Unknown member: {member}")

    if not 0 < test_size < 1:
        raise ValueError("test_size must be a float between 0 and 1.")

    # Shuffle and split only the real data
    random.seed(seed)
    shuffled_real = real_data.copy()
    random.shuffle(shuffled_real)

    split_index = int(len(shuffled_real) * (1 - test_size))
    train_data = shuffled_real[:split_index] + synth_data
    test_data = shuffled_real[split_index:]

    return train_data, test_data

train_data, test_data = train_test_split('acuff')
dataset = Dataset.from_list(train_data)

def formatting_prompts_func(examples):
    convos = examples["messages"]
    
    # Apply chat template without system message and remove newlines
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False,
            system_message=None
        ).replace("\n", "")  # remove all newlines for exact match
        for convo in convos
    ]
    
    return { "text": texts }

dataset = dataset.map(formatting_prompts_func, batched = True,)

Dataset loaded from: /work/users/s/m/smerrill/Albemarle/dataset/kateacuff.txt
Total examples: 321
Dataset loaded from: /work/users/s/m/smerrill/Albemarle/dataset/synth_kateacuff.txt
Total examples: 40


Map: 100%|██████████| 296/296 [00:00<00:00, 16398.07 examples/s]


In [105]:
dataset['text'][0]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>Cutting Knowledge Date: December 2023Today Date: 26 July 2024<|eot_id|><|start_header_id|>user<|end_header_id|>Speaker_1: that we conduct or have staff to evaluateSpeaker_1: our differentiated funding modelSpeaker_1: and see if there are better ways of taking that task on.Speaker_1: Is that a correct or close enough summary of your-Speaker_1: That was much more succinct than I was, Mr. Page.<|eot_id|><|start_header_id|>assistant<|end_header_id|>Okay, any discussion on Dr. Acuff's motion?<|eot_id|>"

### Train

In [107]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=1000,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none"
    ))

#`train_on_completions` method to only train on the assistant 
# outputs and ignore the loss on the user's inputs.
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>",
    response_part = "<|start_header_id|>assistant<|end_header_id|>",
)

Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 296/296 [00:01<00:00, 284.86 examples/s]
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Map (num_proc=128): 100%|██████████| 296/296 [00:01<00:00, 156.74 examples/s]


We verify masking is actually done:

We can see the System and Instruction prompts are successfully masked!

In [108]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 296 | Num Epochs = 28 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,072,204,288 (0.52% trained)


Step,Training Loss
1,1.414600
2,0.953700
3,1.015600
4,0.585000
5,1.193200
6,0.269900
7,0.268500
8,0.297600
9,0.197300
10,0.410700


In [109]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

830.986 seconds used for training.
13.85 minutes used for training.
Peak reserved memory = 31.17 GB.
Peak reserved memory for training = 0.135 GB.
Peak reserved memory % of max memory = 78.921 %.
Peak reserved memory for training % of max memory = 0.342 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [110]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

test_example = [test_data[0]['messages'][0]]
test_answer = test_data[0]['messages'][1]


inputs = tokenizer.apply_chat_template(
    test_example,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nSpeaker_1: Are there any other comments\nSpeaker_1: from any of our school board members?\nSpeaker_1: I'd just like to thank all of our representatives\nKate Acuff: for all the work they do for schools and for getting on a Zoom meeting. I'm sort of with Cree, it is easy to do Zoom in some ways. We may be able to have, we're already talking at the school's level that some of our kids actually like, maybe some of our older kids actually like virtual, but it's still much more fun to see you in person. It is.\nSpeaker_1: My fourth Zoom of the day.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nCutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cuttin

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [111]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer.apply_chat_template(
    test_example,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1., min_p = 0.1)

Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting Cutting 

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [28]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [112]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1., min_p = 0.1)

Describe a tall tower in the capital of France.<|reserved_special_token_172|><|reserved_special_token_217|>assistant<|reserved_special_token_129|>

Describe a tall tower in the capital of France.<|reserved_special_token_121|><|reserved_special_token_122|>assistant<|reserved_special_token_166|>

Describe a tall tower in the capital of France.<|reserved_special_token_176|><|reserved_special_token_20|>assistant<|reserved_special_token_229|>

Describe a tall tower in the capital of France.<|reserved_special_token_140|><|reserved_special_token_61|>assistant<|reserved_special_token_0|>

Describe a tall tower in the capital of France.<|reserved_special_token_244|><|reserved_special_token_72|>assistant<|reserved_special_token_187|>

Describe a tall tower in the capital of France.<|reserved_special_token_106|><|reserved_special_token_206|>assistant<|reserved_special_token_48|>

Describe a tall tower in the capital of France.<|reserved_special_token_50|><|reserved_special_token_177|>assistantила

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [20]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )